**TIP:** This training could take several hours. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

In [ ]:
import pandas as pd 
import re
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM ,SpatialDropout1D
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/AIM/clean_data.csv", lineterminator='\n')
df.head()

,id,txt,dialect
0,1175358310087892992,لكن بالنهاية ينتفض يغير,IQ
1,1175416117793349632,يعني هذا محسوب على البشر حيونه ووحشيه وتطلب...,IQ
2,1175450108898565888,مبين من كلامه خليجي,IQ
3,1175471073770573824,يسلملي مرورك وروحك الحلوه,IQ
4,1175496913145217024,وين هل الغيبه اخ محمد,IQ


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458000 entries, 0 to 457999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458000 non-null  int64 
 1   txt      458000 non-null  object
 2   dialect  458000 non-null  object
dtypes: int64(1), object(2)
memory usage: 10.5+ MB


In [ ]:
X = df['txt']
y = df['dialect']

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
# y= label_encoder.fit_transform(y)
# np.save('/content/gdrive/MyDrive/Colab Notebooks/AIM/dl_classes.npy', label_encoder.classes_)

In [ ]:
# label_encoder.classes_  = np.load('/content/gdrive/MyDrive/Colab Notebooks/AIM/classes.npy', allow_pickle=True)
# y= label_encoder.transform(y)

In [ ]:
voc_size=40000
sent_length=50
embedding_vector_features=50
def _txt_prep(X):
    features = X.to_list()

    onehot_repr=[one_hot(words,voc_size)for words in features] 

    embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
    return embedded_docs

embedded_docs = _txt_prep(X)

In [ ]:
def __create_model():
  model = Sequential()
  model.add(Embedding(voc_size+1, embedding_vector_features, input_length=sent_length))
  model.add(SpatialDropout1D(0.2))
  model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
  model.add(Dense(18, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = __create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            2000050   
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 50)           0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 18)                3618      
                                                                 
Total params: 2,124,468
Trainable params: 2,124,468
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64,callbacks=[cp_callback])

Epoch 1/5
4795/4795 [==============================] - ETA: 0s - loss: 2.0848 - accuracy: 0.3368
Epoch 1: saving model to /content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint
4795/4795 [==============================] - 2225s 463ms/step - loss: 2.0848 - accuracy: 0.3368 - val_loss: 1.8286 - val_accuracy: 0.4218
Epoch 2/5
4795/4795 [==============================] - ETA: 0s - loss: 1.6963 - accuracy: 0.4646
Epoch 2: saving model to /content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint
4795/4795 [==============================] - 2161s 451ms/step - loss: 1.6963 - accuracy: 0.4646 - val_loss: 1.7547 - val_accuracy: 0.4539
Epoch 3/5
4795/4795 [==============================] - ETA: 0s - loss: 1.5150 - accuracy: 0.5253
Epoch 3: saving model to /content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint
4795/4795 [==============================] - 2074s 432ms/step - loss: 1.5150 - accuracy: 0.5253 - val_loss: 1.7533 - val_accuracy: 0.4615
Epoch 4/5


In [ ]:
model.load_weights('/content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint')

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
predict_classes=np.argmax(y_pred,axis=1)

In [ ]:
predict_classes = label_encoder.inverse_transform(predict_classes)
y_test = label_encoder.inverse_transform(y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict_classes))

              precision    recall  f1-score   support

          AE       0.06      0.02      0.03      8827
          BH       0.07      0.04      0.05      8627
          DZ       0.04      0.04      0.04      5498
          EG       0.12      0.14      0.13     18942
          IQ       0.04      0.06      0.05      5082
          JO       0.06      0.04      0.05      9308
          KW       0.10      0.17      0.13     13811
          LB       0.07      0.07      0.07      9035
          LY       0.07      0.07      0.07     12079
          MA       0.02      0.01      0.02      3789
          OM       0.05      0.04      0.05      6211
          PL       0.11      0.06      0.07     14387
          QA       0.06      0.05      0.05     10251
          SA       0.06      0.11      0.08      8938
          SD       0.03      0.05      0.03      4691
          SY       0.04      0.06      0.05      5348
          TN       0.03      0.03      0.03      3018
          YE       0.02    

In [ ]:
import os
import json
from datetime import datetime
from threading import Thread, current_thread, get_ident
from typing import Dict, List, Union
import joblib

report = classification_report(y_test, predict_classes, output_dict=True)
classes = label_encoder.classes_.tolist()
print(classes)
model_status = {}

def _update_status(status: str, classes: List[str] = [], evaluation: Dict = {}) -> None:
    model_status['status'] = status
    model_status['timestamp'] = datetime.now().isoformat()
    model_status['classes'] = classes
    model_status['evaluation'] = evaluation
    with open('model_status.json', 'w+') as file:
        json.dump(model_status, file, indent=2)


_update_status("Model Ready", classes, report)